Источник примера - [BM25 and FAISS hybrid search example](https://gist.github.com/breadchris/b73aae81953eb8f865ebb4842a1c15b5)


In [1]:
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss

class HybridSearch:
    def __init__(self, documents):
        self.documents = documents

        # BM25 initialization
        tokenized_corpus = [doc.split(" ") for doc in documents]
        self.bm25 = BM25Okapi(tokenized_corpus)

        # Sentence transformer for embeddings
        self.model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        self.document_embeddings = self.model.encode(documents)
        
        # FAISS initialization
        self.index = faiss.IndexFlatL2(self.document_embeddings.shape[1])
        self.index.add(np.array(self.document_embeddings).astype('float32'))

    def search(self, query, top_n=10):
        # BM25 search
        bm25_scores = self.bm25.get_scores(query.split(" "))
        top_docs_indices = np.argsort(bm25_scores)[-top_n:]
        
        # Get embeddings of top documents from BM25 search
        top_docs_embeddings = [self.document_embeddings[i] for i in top_docs_indices]
        query_embedding = self.model.encode([query])

        # FAISS search on the top documents
        sub_index = faiss.IndexFlatL2(top_docs_embeddings[0].shape[0])
        sub_index.add(np.array(top_docs_embeddings).astype('float32'))
        _, sub_dense_ranked_indices = sub_index.search(np.array(query_embedding).astype('float32'), top_n)

        # Map FAISS results back to original document indices
        final_ranked_indices = [top_docs_indices[i] for i in sub_dense_ranked_indices[0]]

        # Retrieve the actual documents
        ranked_docs = [self.documents[i] for i in final_ranked_indices]

        return ranked_docs


RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [2]:
# Sample usage
documents = [
    "Artificial Intelligence is changing the world.",
    "Machine Learning is a subset of AI.",
    "Deep Learning is a subset of Machine Learning.",
    "Natural Language Processing involves understanding text.",
    "Computer Vision allows machines to see and understand.",
    "AI includes areas like NLP and Computer Vision.",
    "The Pyramids of Giza are architectural marvels.",
    "Mozart was a prolific composer during the classical era.",
    "Mount Everest is the tallest mountain on Earth.",
    "The Nile is one of the world's longest rivers.",
    "Van Gogh's Starry Night is a popular piece of art."
    "Basketball is a sport played with a round ball and two teams."
]

In [4]:
hs = HybridSearch(documents)
query = "Tell me about AI in text and vision."
results = hs.search(query, top_n=10)
print(results)

['AI includes areas like NLP and Computer Vision.', 'Machine Learning is a subset of AI.', 'Computer Vision allows machines to see and understand.', 'Natural Language Processing involves understanding text.', "Van Gogh's Starry Night is a popular piece of art.Basketball is a sport played with a round ball and two teams.", 'Deep Learning is a subset of Machine Learning.', 'Mozart was a prolific composer during the classical era.', "The Nile is one of the world's longest rivers.", 'The Pyramids of Giza are architectural marvels.', 'Mount Everest is the tallest mountain on Earth.']


In [ ]:
documents = [
    "Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈",
    "⭐️  Кнопка: ⭐️START⭐️(https://t.me/major/start?startapp=1972869792)",
    "Таро-прогноз на 23 Июля – Туз Мечей.   🔮 Совет: Меч – это достаточное количество сил для энергичных начинаний, огромная решимость действовать, начало успешной борьбы, готовность к росту, отрыву от изначального существования. Это ситуация, когда сам человек для себя многое прояснил и знает теперь, что хочет делать дальше. Это показатель силы – физической силы, силы воли, силы, приобретенной положением и умом или в силу обстоятельств. Триумф личной силы, власти над обстоятельствами. Триумф может относиться к любой стороне жизни: к работе, любви, денежным делам, духу, любым увлекающим занятиям. Этот Туз не столько начало, но и показатель завоеванного, торжества провозглашенных взглядов и принятых решений.   💰 Деньги: Процветание. Принятие однозначных и окончательных решений в этих вопросах. Возможность улучшить качество жизни, приняв вызов.   🩷 Любовь: В области отношений Туз Мечей символизирует импульсивную первобытную силу, интенсивные «завоевательные» эмоции, крайние чувства, связанные с ситуацией или человеком. Эмоции, которым покровительствует Туз Мечей, способны воспламенить таким огнем, который сожжет все препятствия на пути к цели, попутно причинив немало вреда. Мечи вообще масть холодная, и когда на горизонте в кои-то веки появляется единственная эмоция, то она заполоняет собой всё со свойственной этой масти тотальностью.  🎁🔥 РАСПРОДАЖА КУРСА «ОРАКУЛ ЛЕНОРМАН. БАЗОВЫЙ КУРС» В РАССРОЧКУ 👉🏻 http://alexeygrishin.com/lenorman_base.",
    "он вообще не собирается переезжать в другое государство",
    "ты не мог бы набрать меня после обеда"
]

In [ ]:
hs = HybridSearch(documents)
query = "страна"
results = hs.search(query, top_n=10)
print(results)

Объеснениее кода
====

Этот код реализует **гибридную систему поиска**, сочетающую методы **BM25** (статистический подход) и **векторного поиска** (семантический подход) для повышения релевантности результатов. Вот детальный разбор:

---

### **1. Инициализация (`__init__`)**  
- **BM25Okapi**  
  - Токенизирует корпус документов (разделяет текст на слова) .  
  - Инициализирует BM25 для ранжирования документов по ключевым словам.  

- **SentenceTransformer**  
  - Создает семантические эмбеддинги документов с помощью модели `paraphrase-MiniLM-L6-v2` .  
  - Эти эмбеддинги отражают смысл текста, а не только ключевые слова.  

- **FAISS**  
  - Индексирует эмбеддинги документов для быстрого поиска по сходству (используется L2-расстояние) .  

---

### **2. Поиск (`search`)**  
- **Этап 1: BM25**  
  - Вычисляет релевантность документов по ключевым словам из запроса .  
  - Возвращает `top_n` документов с наивысшими BM25-баллами.  

- **Этап 2: Векторный поиск**  
  - Берет эмбеддинги отобранных BM25 документов и запроса.  
  - Использует FAISS для поиска семантически ближайших документов .  

- **Финальное ранжирование**  
  - Сопоставляет результаты FAISS с исходными индексами документов.  

---

### **Пример работы**  
Для запроса *"Tell me about AI in text and vision"* система:  
1. Найдет документы с ключевыми словами **AI**, **text**, **vision** (BM25).  
2. Переотранжирует их по семантической близости, учитывая смысл (например, документы про NLP и Computer Vision получат приоритет) .  

---

### **Преимущества гибридного подхода**  
- **BM25** быстро отбирает кандидаты по ключевым словам.  
- **SentenceTransformer + FAISS** уточняют результаты, учитывая контекст и смысл .  

---
